In [2]:
#Imports
import time
import requests
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup
# !pip install selenium
# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options
import os
import urllib.request
from urllib.request import urlopen
import numpy as np
import pandas as pd
from pandas import DataFrame
# import pyautogui
from requests.exceptions import ConnectionError

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
internal_urls = set()
external_urls = set()

In [5]:
def is_url_valid(url):
   #Checks if the url is valid or not
    parsedURL = urlparse(url)
    return bool(parsedURL.netloc) and bool(parsedURL.scheme)

#The above function works as follows. We check if the scheme is present and there is a value in the network location part
# url = "https://umd.edu/virusinfo"
# urlparse(url)
# ParseResult(scheme='https', netloc='umd.edu', path='/virusinfo', 
# params='', query='', fragment='')

In [6]:
#The function gives all urls
count = 0
def get_all_urls(url):
    urls = set()
    # domain name of the URL without the protocol (umd.edu in this case)
    domain_name = urlparse(url).netloc
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    for a_tag in soup.findAll("a"):
        href = a_tag.attrs.get("href")
        if href == "" or href is None:
            #href is empty and we don't need that a element
            continue
        #if the link is not absolute, make it by joining relative to the base
        href = urljoin(url, href)
        parsed_href = urlparse(href)
        #constructing an absolute URL from parsed data
        href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path
        if not is_url_valid(href):
            #in valid url
            continue
        if href in internal_urls:
            #it is already in the set, so we don't need to add
            continue
        if domain_name not in href:
            #it is an external link. i.e
            # Check if it is already there 
            if href not in external_urls:
#                 print(f"[EXT] External link: {href}")
                external_urls.add(href)
            continue
#         print(".",end=" ")
        global count
        count = count+1
        print(f"{count}[INT] Internal link: {href}")

        urls.add(href)
        internal_urls.add(href)
    return urls

In [7]:
total_urls_visited = 0
def crawl(url, max_urls=30000):
    #Max URL is just to decrease the time if there are a lot of pages.
    #The following code was openly available of github and I found this
    #idea useful to inhibit crawling time
    global total_urls_visited
    total_urls_visited += 1
    try:
        links = get_all_urls(url)
        for link in links:
            if total_urls_visited > max_urls:
                break
            crawl(link, max_urls=max_urls)
    except Exception as e:
        print(e)

In [8]:
#base_url = "https://umd.edu/virusinfo"
#Input
# base_url = input("Enter the URL : ")
base_url = "https://www.archanaskitchen.com/"
internal_urls.add(base_url)

In [9]:
parsedurl = urlparse(base_url)
print(parsedurl)
base_url_main = parsedurl.scheme+"://"+parsedurl.netloc
print(base_url_main)
# base_url_text = base_url.split("//",1)[1]
base_url_text = parsedurl.netloc+parsedurl.path
print(base_url_text)
# base_url_text_domain = base_url_text.split("/",1)[0]
base_url_text_domain = parsedurl.netloc
print(base_url_text_domain)

ParseResult(scheme='https', netloc='www.archanaskitchen.com', path='/', params='', query='', fragment='')
https://www.archanaskitchen.com
www.archanaskitchen.com/
www.archanaskitchen.com


In [ ]:
crawl(base_url)
print("[+] Total External links:", len(external_urls))
print("[+] Total Internal links:", len(internal_urls))
print("[+] Total:", len(external_urls) + len(internal_urls))

1[INT] Internal link: https://www.archanaskitchen.com/about-archana
2[INT] Internal link: https://www.archanaskitchen.com/contact-us
3[INT] Internal link: https://www.archanaskitchen.com/advertise
4[INT] Internal link: https://www.archanaskitchen.com/archanas-kitchen-partners
5[INT] Internal link: https://www.archanaskitchen.com/brands
6[INT] Internal link: https://www.archanaskitchen.com/media-coverage
7[INT] Internal link: https://www.archanaskitchen.com/recipes
8[INT] Internal link: https://www.archanaskitchen.com/recipes/snack-recipes
9[INT] Internal link: https://www.archanaskitchen.com/recipes/snack-recipes/indian-snack-recipes
10[INT] Internal link: https://www.archanaskitchen.com/recipes/snack-recipes/homemade-energy-bar-recipes
11[INT] Internal link: https://www.archanaskitchen.com/recipes/snack-recipes/sandwich-recipes
12[INT] Internal link: https://www.archanaskitchen.com/recipes/snack-recipes/burger-recipes
13[INT] Internal link: https://www.archanaskitchen.com/recipes/snac

182[INT] Internal link: https://www.archanaskitchen.com/tag/vegetarian-recipes
183[INT] Internal link: https://www.archanaskitchen.com/tag/side-dish-recipes
184[INT] Internal link: https://www.archanaskitchen.com/tag/party-appetizer-recipes
185[INT] Internal link: https://www.archanaskitchen.com/tag/cabbage-recipes
186[INT] Internal link: https://www.archanaskitchen.com/eggplant-steaks-recipe
187[INT] Internal link: https://www.archanaskitchen.com/tag/party-food-recipes
188[INT] Internal link: https://www.archanaskitchen.com/tag/healthy-recipes
189[INT] Internal link: https://www.archanaskitchen.com/tag/chickpea-recipes
190[INT] Internal link: https://www.archanaskitchen.com/tag/eggplant-brinjal-baingan-recipes
191[INT] Internal link: https://www.archanaskitchen.com/tag/antioxidant-rich-recipes
192[INT] Internal link: https://www.archanaskitchen.com/paneer-filo-pastry-pie
193[INT] Internal link: https://www.archanaskitchen.com/tag/paneer-recipes
194[INT] Internal link: https://www.arch

312[INT] Internal link: https://www.archanaskitchen.com/spinach-watermelon-salad-recipe-with-walnuts-in-hindi
313[INT] Internal link: https://www.archanaskitchen.com/raw-mango-cucumber-salad-recipe-with-roasted-peanuts-in-hindi
314[INT] Internal link: https://www.archanaskitchen.com/classic-greek-salad-recipe-in-hindi
315[INT] Internal link: https://www.archanaskitchen.com/cuisine/continental
316[INT] Internal link: https://www.archanaskitchen.com/apple-cucumber-minty-salad-recipe
317[INT] Internal link: https://www.archanaskitchen.com/tag/summer-recipe-ideas
318[INT] Internal link: https://www.archanaskitchen.com/tag/apple-recipes
319[INT] Internal link: https://www.archanaskitchen.com/tag/cucumber-recipes
320[INT] Internal link: https://www.archanaskitchen.com/chilled-sweet-sour-cucumber-noodle-salad-recipe
321[INT] Internal link: https://www.archanaskitchen.com/yummy-creamy-spinach-pesto-penne-pasta-recipe-kids-recipes-made-with-del-monte
322[INT] Internal link: https://www.archanas

398[INT] Internal link: https://www.archanaskitchen.com/quick-bun-pizza
399[INT] Internal link: https://www.archanaskitchen.com/beetroot-salad-recipe-with-mixed-greens-feta-cheese
400[INT] Internal link: https://www.archanaskitchen.com/carrot-ribbon-salad-with-asian-sesame-dressing-recipe
401[INT] Internal link: https://www.archanaskitchen.com/smoked-tofu-and-grilled-vegetable-salad
402[INT] Internal link: https://www.archanaskitchen.com/lentil-tabbouleh-recipe-middle-eastern-vegetarian-salad-with-lentils
403[INT] Internal link: https://www.archanaskitchen.com/cuisine/mediterranean
404[INT] Internal link: https://www.archanaskitchen.com/tag/high-protein-diet
405[INT] Internal link: https://www.archanaskitchen.com/tag/mango-recipes
406[INT] Internal link: https://www.archanaskitchen.com/tag/raw-mango-recipes
407[INT] Internal link: https://www.archanaskitchen.com/tag/couscous-recipes
408[INT] Internal link: https://www.archanaskitchen.com/sweet-and-spicy-raw-mango-chutney
409[INT] Inter

499[INT] Internal link: https://www.archanaskitchen.com/banana-almond-milkshake-smoothie
500[INT] Internal link: https://www.archanaskitchen.com/how-to-make-homemade-yogurt-curd
501[INT] Internal link: https://www.archanaskitchen.com/tag/yogurt-dahi-recipes
502[INT] Internal link: https://www.archanaskitchen.com/tag/healthy-kids-snacks
503[INT] Internal link: https://www.archanaskitchen.com/tag/strawberry-recipes
504[INT] Internal link: https://www.archanaskitchen.com/pomegranate-and-chia-seeds-yogurt-smoothie-recipe
505[INT] Internal link: https://www.archanaskitchen.com/healthy-delicious-jamun-fruit-smoothie-black-plum-smoothie-recipe
506[INT] Internal link: https://www.archanaskitchen.com/chewy-strawberry-muesli-oats-cookie-recipe
507[INT] Internal link: https://www.archanaskitchen.com/muesli-oats-cupcake-filled-with-yogurt-and-strawberries-recipe
508[INT] Internal link: https://www.archanaskitchen.com/whole-wheat-muesli-pancake-with-spirulina-recipe
509[INT] Internal link: https://

...

16572[INT] Internal link: http://www.archanaskitchen.com/gongura-pappu-gongura-sorrel-leaves-dal
16573[INT] Internal link: http://www.archanaskitchen.com/kashmiri-paneer-gravy-recipe
16574[INT] Internal link: http://www.archanaskitchen.com/a-delicious-collection-of-hariyali-indian-non-veg-recipes-you-must-try
16575[INT] Internal link: https://www.archanaskitchen.com/a-delicious-collection-of-hariyali-indian-non-veg-recipes-you-must-try
16576[INT] Internal link: http://www.archanaskitchen.com/hariyali-machali-recipe-carp-fish-in-green-spinach-and-coriander-gravy
16577[INT] Internal link: http://www.archanaskitchen.com/kasuri-methi-mach-recipe-dry-fenugreek-flavoured-fish-curry
16578[INT] Internal link: http://www.archanaskitchen.com/hariyali-mutton-curry-recipe
16579[INT] Internal link: http://www.archanaskitchen.com/hariyali-chicken-tikka-masala-recipe
16580[INT] Internal link: http://www.archanaskitchen.com/karuvepilai-chicken-fry-recipe
16581[INT] Internal link: http://www.archanaski

16721[INT] Internal link: http://www.archanaskitchen.com/kadhi-bari-recipe
16722[INT] Internal link: http://www.archanaskitchen.com/chanar-dalna-recipe-in-hindi
16723[INT] Internal link: http://www.archanaskitchen.com/jhure-jhure-aloo-bhaja-recipe-in-hindi
16724[INT] Internal link: http://www.archanaskitchen.com/bengali-cholar-dal-recipe-in-hindi
16725[INT] Internal link: http://www.archanaskitchen.com/bengali-mooli-aloo-ki-sabzi-in-mustard-gravy-recipe
16726[INT] Internal link: http://www.archanaskitchen.com/aloo-parwal-in-poppy-seed-masala-recipe-in-hindi
16727[INT] Internal link: http://www.archanaskitchen.com/bengali-style-chingri-posto-recipe-prawns-cooked-in-poppy-seed-gravy
16728[INT] Internal link: http://www.archanaskitchen.com/bengali-style-cholar-dal-recipe-in-hindi-2
16729[INT] Internal link: http://www.archanaskitchen.com/bengali-style-cholar-dal-recipe-in-hindi
16730[INT] Internal link: https://www.archanaskitchen.com/bengali-style-cholar-dal-recipe-in-hindi-2
16731[INT] 

16806[INT] Internal link: http://www.archanaskitchen.com/kerala-style-vettu-cake-recipe-eggless-wholewheat-tea-shop-cake
16807[INT] Internal link: http://www.archanaskitchen.com/tag/canapes-recipes
16808[INT] Internal link: http://www.archanaskitchen.com/cheese-stuffed-mashed-potato-balls-recipe-non-fried
16809[INT] Internal link: http://www.archanaskitchen.com/pan-fried-rice-balls-filled-with-cheese-recipe
16810[INT] Internal link: http://www.archanaskitchen.com/cheesy-rice-poppers
16811[INT] Internal link: http://www.archanaskitchen.com/chicken-parmigiania-recipe
16812[INT] Internal link: https://www.archanaskitchen.com/kerala-style-vettu-cake-recipe-eggless-wholewheat-tea-shop-cake
16813[INT] Internal link: http://www.archanaskitchen.com/kandarappam-recipe-chettinad-sweet-appam
16814[INT] Internal link: http://www.archanaskitchen.com/avocado-ada-pradhaman-recipe-with-vegan-option
16815[INT] Internal link: http://www.archanaskitchen.com/thatta-payaru-sambar-recipe-black-eyed-beans-sa

16885[INT] Internal link: http://www.archanaskitchen.com/huruli-kaalu-chutney-pudi-recipe-horse-gram-chutney-podi
16886[INT] Internal link: http://www.archanaskitchen.com/horsegram-masala-dosa-recipe
16887[INT] Internal link: http://www.archanaskitchen.com/kollu-sambar-recipe-horsegram-sambar-with-vegetables
16888[INT] Internal link: http://www.archanaskitchen.com/kashmiri-kulith-ki-dal-recipe-horse-gram-dal-recipe
16889[INT] Internal link: http://www.archanaskitchen.com/himachal-pahari-style-teliah-mah-recipe-brown-horse-gram-curry
16890[INT] Internal link: http://www.archanaskitchen.com/ulava-charu-recipe-horse-gram-stew
16891[INT] Internal link: http://www.archanaskitchen.com/portion-control-meal-drumstick-leaves-sambar-vazha-thandu-poriyal-avarakki-poricha-kootu-salad-curd
16892[INT] Internal link: https://www.archanaskitchen.com/huruli-kaalu-chutney-pudi-recipe-horse-gram-chutney-podi
16893[INT] Internal link: http://www.archanaskitchen.com/andhra-style-nalla-karam-podi-recipe
168

16968[INT] Internal link: http://www.archanaskitchen.com/sev-usal-recipe
16969[INT] Internal link: http://www.archanaskitchen.com/kathirikai-gothsu-recipe-spicy-and-tangy-eggplant-curry-in-hindi
16970[INT] Internal link: http://www.archanaskitchen.com/kathirikai-gothsu-spicy-and-tangy-eggplant-curry
16971[INT] Internal link: http://www.archanaskitchen.com/baingan-masala-recipe-in-hindi
16972[INT] Internal link: http://www.archanaskitchen.com/lahsuni-palak-dal-recipe-spinach-lentil-curry-with-garlic
16973[INT] Internal link: http://www.archanaskitchen.com/dal-bukhara-recipe-punjabi-style-black-urad-dal-in-hindi
16974[INT] Internal link: http://www.archanaskitchen.com/tadkewali-masoor-dal
16975[INT] Internal link: http://www.archanaskitchen.com/palak-onion-kadhi-pakora-recipe
16976[INT] Internal link: http://www.archanaskitchen.com/tadkewali-masoor-dal-recipe-high-protein-masoor-dal-recipe-in-hindi
16977[INT] Internal link: http://www.archanaskitchen.com/navrang-dal-recipe-spicy-mixed-le

In [ ]:
len(internal_urls)

In [29]:
setInternalUrls = set(internal_urls)
len(setInternalUrls)

22235

In [30]:
df = DataFrame(internal_urls,columns=['URL'])

In [31]:
df.shape

(22235, 1)

In [32]:
dfrecipe = df[df['URL'].str.contains('recipe')]

In [33]:
dfrecipe

,URL
0,https://www.archanaskitchen.com/masala-karela-...
1,http://www.archanaskitchen.com/spicy-tomato-ri...
3,http://www.archanaskitchen.com/ragi-vermicelli...
4,http://www.archanaskitchen.com/tag/kids-lunch-...
5,http://www.archanaskitchen.com/gongura-chicken...
...,...
22226,https://www.archanaskitchen.com/kerala-cheriya...
22227,http://www.archanaskitchen.com/pav-bhaji-flavo...
22230,https://www.archanaskitchen.com/tag/chilli-rec...
22231,https://www.archanaskitchen.com/keerai-masiyal...


In [34]:
dfrecipe.to_csv('urls_recipe10k.csv')